# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/cities.csv"
df = pd.read_csv(csv_path)
df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,omboue,-1.5746,9.2618,74.59,79,83,8.30,GA,1629773706
1,saint anthony,45.0205,-93.2180,80.20,64,1,1.99,US,1629773120
2,beloha,-25.1667,45.0500,61.92,94,0,4.63,MG,1629773185
3,ribeira grande,38.5167,-28.7000,72.00,100,75,10.36,PT,1629773116
4,sorland,67.6670,12.6934,51.10,83,100,15.88,NO,1629773111
5,carnarvon,-24.8667,113.6333,73.47,48,0,14.97,AU,1629773706
6,hobart,-42.8794,147.3294,54.05,61,20,3.00,AU,1629773546
7,ahuimanu,21.4447,-157.8378,80.31,75,90,13.80,US,1629773189
8,mayumba,-3.4320,10.6554,74.05,85,95,9.15,GA,1629773707
9,somerton,32.5964,-114.7097,99.50,19,1,9.22,US,1629773707


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key = g_key)

coordinates = df[['Lat', 'Lng']]
humidity = df['Humidity']

fig = gmaps.figure(center=(0,-10),zoom_level=2,layout={'width': '100%','height':'475px'})
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 2)
fig.add_layer(heat_layer)
plt.savefig('../output_data/fig22.png')
fig


Figure(layout=FigureLayout(height='475px', width='100%'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = df.loc[(df['Max Temp']< 80) & (df['Max Temp']> 70) & (df['Wind Speed']< 10) & (df['Cloudiness']==0)].dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,shubarshi,48.5872,57.1869,76.37,29,0,8.86,KZ,1629773718,
99,dubrovnik,42.6481,18.0922,74.84,38,0,5.75,HR,1629773729,
106,weiser,44.2510,-116.9693,76.32,18,0,9.95,US,1629773730,
243,barcelos,41.5388,-8.6151,77.29,49,0,3.62,PT,1629773765,
289,puerto quijarro,-17.7833,-57.7667,77.68,34,0,1.83,BO,1629773778,
297,nalut,30.3333,10.8500,76.50,37,0,9.62,LY,1629773781,
311,sabha,27.0377,14.4283,77.72,33,0,9.17,LY,1629773784,
319,jalu,29.0331,21.5482,76.51,45,0,8.30,LY,1629773786,
325,cody,44.5263,-109.0565,72.63,39,0,3.00,US,1629773769,
337,methoni,36.8197,21.7049,76.21,65,0,3.33,GR,1629773791,


In [8]:
hotel_params = {
    "radius": 5000,
    "type": 'lodging',
    'key': g_key   
}
place_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    hotel_params['location'] = f"{row['Lat']}, {row['Lng']}"
    
    results = requests.get(place_url, params=hotel_params).json()
    try:
        hotel_df.loc[index, 'Hotel Name']=results['results'][1]['name']
    except(IndexError,KeyError):
        print('Missing field/results... skipping')
hotel_df        

Missing field/results... skipping
Missing field/results... skipping
Missing field/results... skipping


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,shubarshi,48.5872,57.1869,76.37,29,0,8.86,KZ,1629773718,
99,dubrovnik,42.6481,18.0922,74.84,38,0,5.75,HR,1629773729,Hotel Bellevue Dubrovnik
106,weiser,44.2510,-116.9693,76.32,18,0,9.95,US,1629773730,State Street Motel
243,barcelos,41.5388,-8.6151,77.29,49,0,3.62,PT,1629773765,Hotel Restaurante Bagoeira
289,puerto quijarro,-17.7833,-57.7667,77.68,34,0,1.83,BO,1629773778,
297,nalut,30.3333,10.8500,76.50,37,0,9.62,LY,1629773781,
311,sabha,27.0377,14.4283,77.72,33,0,9.17,LY,1629773784,محطة ام الارانب -سبها مواصلات
319,jalu,29.0331,21.5482,76.51,45,0,8.30,LY,1629773786,Jalu Hotel
325,cody,44.5263,-109.0565,72.63,39,0,3.00,US,1629773769,The Cody Hotel
337,methoni,36.8197,21.7049,76.21,65,0,3.33,GR,1629773791,ILIAS APARTMENTS


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig = gmaps.figure(center=(0,-10),zoom_level=2,layout={'width': '100%','height':'475px'})
fig.add_layer(heat_layer)
fig.add_layer(markers)

In [11]:
fig

Figure(layout=FigureLayout(height='475px', width='100%'))